In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [3]:
path = "/home/kalyan/DataSets/animals/"

In [4]:
print(os.listdir(path))

['trainbeg.npy', 'testbeg.npy', 'trainLabels.npy', 'test.csv', 'train', 'train.csv', 'test', 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'sample_submission.csv']


In [5]:
X_train = np.load(path + "trainbeg.npy")

In [6]:
X_test = np.load(path + "testbeg.npy")

In [7]:
Y_train =  np.load(path + "trainLabels.npy")
print(Y_train.shape)
Y_train = Y_train.reshape(Y_train.shape[0])
np.squeeze(Y_train)
print(Y_train.shape)
print(Y_train)

(13000, 1)
(13000,)
[11 26 10 ... 26 26 26]


Resnet Architecture

In [8]:
from keras.applications import ResNet50
from keras.models import Sequential, load_model, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2023-05-09 10:03:18.607634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 10:03:19.429711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense
from keras import regularizers
resnet_weights = "/home/kalyan/gitrepo/AI-ML-Playground/Animal_Streamlit/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = ResNet50(weights = resnet_weights, include_top=False, input_shape=(64, 64, 3))

# Add a dense layer on top of the ResNet50 model
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
predictions = Dense(30, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [12]:
model.compile(optimizer=Adam(lr=0.0001),
                loss='categorical_crossentropy',        
                metrics=['accuracy'])


/home/kalyan/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
model.fit(X_train/255., to_categorical(Y_train, num_classes=30), batch_size=64, epochs=10, verbose=1)

Epoch 1/10


2023-05-09 10:03:49.039073: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.


 12/204 [>.............................] - ETA: 5:55 - loss: 17.1988 - accuracy: 0.0846

KeyboardInterrupt: 

In [ ]:
model.save(path + "self_resnet50.h5")

In [ ]:
#predicting on test data
y_test_predict = model.predict(X_test/255.)
print(y_test_predict)


188/188 [==============================] - 6s 22ms/step
[[5.6558970e-06 4.6858077e-05 1.1881919e-02 ... 4.2412494e-04
  7.0931906e-06 1.4301834e-06]
 [4.6921517e-03 3.3467944e-04 3.2227337e-03 ... 2.0731888e-03
  1.3556154e-03 3.0760103e-04]
 [5.7905918e-01 1.1395101e-03 7.7154045e-03 ... 5.2771047e-03
  1.3814449e-01 1.6585598e-04]
 ...
 [2.9387014e-05 5.5083847e-01 9.4704889e-03 ... 8.3074541e-05
  6.5276829e-05 5.8108318e-04]
 [9.6382053e-07 1.7766629e-07 9.8776954e-06 ... 2.0847212e-08
  9.4356061e-08 6.3748342e-07]
 [2.5379664e-04 7.2680326e-05 7.8604020e-02 ... 1.0547791e-04
  1.4336433e-04 1.3985486e-04]]
